## You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/tutorials/03_deeppavlov_gobot.ipynb).

In [1]:
import copy
from pprint import pprint

In [ ]:
!pip3 install deeppavlov

In [4]:
import deeppavlov

**NOTE**: "go_bot" model trains faster on a CPU, so let's ignore existing GPUs:

In [5]:
!export CUDA_VISIBLE_DEVICES=""
!echo "cuda visible devices = '"$CUDA_VISIBLE_DEVICES"'"

cuda visible devices = ''


# Hybrid goal-oriented bot

Dialog bots are categorized into two types:

1. **goal-oriented models **

    (those who have to achieve some kind of a goal in the end of conversation: 
     - restaurant and flight booking,
     - customer support service,
     - etc.);
 
2. **chit-chat models **

   (those who chat just for fun, the longer bot speaks with you the better, example:
    - "replica" mobile application).

We will only dive into goal-oriented task specification.

![go bot architecture 00](img/bot_architecture00.png)

A classical dialog system consists of:

1. **Natural Language Understanding component (NLU)**

 that is intended to "understand" human and represent it's "understanding" in a machine readable format. 

 It takes an utterance text as input and converts to a dialog "frame".

 "Frame" may consist of:
   - domain value (domain is some kind of "a type of dialogs");
   - intent value (intent of current human utterance: "welcome_message", "asking_weather", etc.);
   - entity slots (entities are mentioned by human "location", "time", etc.).

2. **Dialogue Manager component (DM)**

 that is intended to decide what to respond. 
    
 It takes a filled by NLU frame and outputs action (it isn't a final text, it is a label). 
    
 For example, there may be actions: "say_welcome", "say_goodbye", "ask_location", "give_weather", etc.

3. **Natural Language Generation component(NLG)**

 that is intended to convert action to an actual text response representation.
 
 For example, "say_goodbye" -> "You are welcome!".

## NLU 

![go bot architecture 01](img/bot_architecture01.png)

Let's consider a dialog system with NLU component that consists of a single Named Entity Recognition 
    component (or NER).

One of the previous tutorials introduced deeppavlov NER model and showed how to use it.

## DM & NLG

The tutorial is focused on how to implement
 - Dialogue Manager and
 - Natural Language Generator.

### Dataset

We will train the chatbot on a [Dialog State Tracking Chellenge 2](http://camdial.org/~mh521/dstc/) data.

Let's download it first.

In [6]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader

data = DSTC2DatasetReader().read(data_path="tmp/my_download_of_dstc2")

2018-11-14 15:47:22.520 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 96: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to tmp/my_download_of_dstc2]
2018-11-14 15:47:22.524 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-14 15:47:22.528 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz HTTP/1.1" 200 506300
2018-11-14 15:47:22.529 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to tmp/my_download_of_dstc2/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 9.11MB/s]
2018-11-14 15:47:22.586 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting tmp/my_download_of_dstc2/dstc2_v2.tar.gz archive into tmp/my_download_of_dstc2
2018-11-14 15:47:22.608 INFO in 'deeppavlov.dataset_reader

`DSTC2DatasetReader` downloaded the needed data and saved to disk.

`DialogDatasetIterator` took the data as input and transformed it to batches.

In [7]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

batches_generator = DialogDatasetIterator(data, seed=1443, shuffle=True)\
                                         .gen_batches(batch_size=4, data_type='train')

-------------
-------------
   Let's take a closer look at a batch content:

In [8]:
batch = batches_generator.__next__()

Each batch is a tuple of two elements:
  - list of x's and
  - list of y's

In [9]:
x_batch, y_batch = batch

`x_batch` (and `y_batch`) consists of 4 samples. This is because `batch_size` was 4.

In [10]:
len(x_batch)

4

One sample is a dialog. This is how one turn from a dialog looks like:

In [11]:
dialog_id = 0
dialog = [(x, y) for x, y in zip(x_batch[dialog_id], y_batch[dialog_id])]

turn_id = 0
print("----{}th turn----".format(turn_id)) 
pprint(dialog[turn_id], indent=8)

----0th turn----
(       {'intents': [], 'prev_resp_act': None, 'text': ''},
        {       'act': 'welcomemsg',
                'text': 'Hello, welcome to the Cambridge restaurant system. '
                        'You can ask for restaurants by area, price range or '
                        'food type. How may I help you?'})


This is how the whole dialog looks (printing `'text'` parts of `x_batch` and `y_batch`):

In [12]:
for turn in dialog:
    x, y = turn
    print('::', x['text']) 
    print('>>', y['text'], '\n')

:: 
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you? 

:: traditional
>> api_call area="dontcare" food="traditional" pricerange="dontcare" 

:: traditional
>> I am sorry but there is no traditional restaurant that matches your request. 

:: italian food
>> What part of town do you have in mind? 

:: south
>> api_call area="south" food="italian" pricerange="dontcare" 

:: south
>> Pizza hut cherry hinton is a nice place in the south of town serving tasty italian food. 

:: phone number
>> The phone number of pizza hut cherry hinton is 01223 323737. 

:: post code
>> The post code of pizza hut cherry hinton is C.B 1, 7 D.Y. 

:: thank you good bye
>> You are welcome! 



### HowTo: DeepPavlov configs

DeepPavlov uses json configs as a tool to configure data pipelines.

Config has the following sections:
     
 - **dataset_reader**
   - configuration of dataset reader component (is responsible for data download and saving to disk);
  
 - **dataset_iterator**
   - configuration of dataset iterator component (is responsible for making batches (sequences) of data that will be further fed to pipe components);
  
 - **metadata**
   - extra info (urls for data download and telegram configuration);

 - **train**
   - training process configuration (size of batches, number of training epochs, etc.);
   
 - **chainer**
   - specifies data flow (which components are run and in what order);

Let's construct a simple config that builds a dictionary of input sample tokens.

In [13]:
vocab_config = {}

- **dataset_reader** -- configuration of dataset reader component (that is responsible for data download and saving to disk)

In [14]:
dstc2_reader_comp_config = {
    'class_name': 'dstc2_reader',
    'data_path': 'dstc2'
}

In [15]:
vocab_config['dataset_reader'] = dstc2_reader_comp_config

- **dataset_iterator** -- configuration of dataset iterator component (that is responsible for making batches (sequences) of data that will be further fed to pipe components)

In [16]:
dialog_iterator_comp_config = {
    'class_name': 'dialog_iterator'
}

In [17]:
vocab_config['dataset_iterator'] = dialog_iterator_comp_config

- **metadata** -- some extra info
     - **metadata.download** -- a list of data which should be downloaded in order for config to work

In [18]:
dstc2_download_config = {
    'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2'
}

In [19]:
vocab_config['metadata'] = {}
vocab_config['metadata']['download'] = [
    dstc2_download_config
]

- **train** -- training process configuration
     
We don't need to train anything now, just build (fit on whole dataset once) a dictionary, so "train" only specifies that metrics are empty.

In [20]:
vocab_config['train'] = {
    'validate_best': False,
    'test_best': False
}

 - **chainer** specifies data flow:
 
     - **chainer.in** -- is a list of input sample names (one data sample might consist of several variables);
     - **chainer.in_y** -- is a list of input label names (each sample might have labels of different kind);
     - **chainer.out** -- is a list of output prediction names (usually has the same length as "chainer.in_y");
     
X is only an utterance here.

Y is empty (we don't need to train the dictionary like neural networks)

There is no prediction for the config, nothing to predict.

In [21]:
vocab_config['chainer'] = {}
vocab_config['chainer']['in'] = ['utterance']
vocab_config['chainer']['in_y'] = []
vocab_config['chainer']['out'] = []

- **chainer**
     - **chainer.pipe** -- is a list of consequently run components. This is the place where you specify in which order and what kind of data will be fed to components. 
     
Our pipe consists of one component -- "default_vocab".

##### HowTo: Component config

Component configs are always just a part of a global model config (described above).

Config for any component contains the following **_required_** parameters:
 - **class_name** -- registered name of a component (it is a link to python component implementation)
 - **save_path** -- path to save the component (sometimes is not needed, for example, for tokenizers)
 - **load_path** -- path to load the component (sometimes is not needed, for examples, for tokenizers)

and the following **_optional_** parameters:
 - **id** -- reference name for the component
 - **ref** -- "id" of a component that was previously initialized. It can be used instead of "name".
 - **fit\_on** -- a list of data fields to fit on (it calls \_\_fit\_\_ method of the component)
 - **in** -- a list of data fields that are inputs during inference (prediction)
 - **out** -- a list of data fields that are outputs during inference (prediction)
 
 
"default_vocab" component also has it's on unique parameters:
 - level -- on which level to operate ('token' level and 'char' (character) level are available)
 - tokenizer -- if input is a string, then it will be tokenized by the tokenizer, _optional parameter_

In [22]:
vocab_comp_config = {
    'class_name': 'default_vocab',
    'save_path': 'vocabs/token.dict',
    'load_path': 'vocabs/token.dict',
    'fit_on': ['utterance'],
    'level': 'token',
    'tokenizer': {
        'class_name': 'split_tokenizer'
    },
    'main': True
}

In [23]:
vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]

To download "dstc2_v2" dataset use `deeppavlov.deep_download` script (you have to do it only once):

In [24]:
from deeppavlov import deep_download # it is called "deep" in honor of "Deep Pavlov"

deep_download(vocab_config)

2018-11-14 15:47:37.246 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-14 15:47:37.250 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz HTTP/1.1" 200 506300
2018-11-14 15:47:37.251 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 12.9MB/s]
2018-11-14 15:47:37.291 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2_v2.tar.gz archive into /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2


All data and models are saved to `./dstc2`.

In [27]:
dstc2_path = './dstc2'

Data was downloaded to `dstc2_path`:

In [28]:
# The command will only work for linux, do not panic otherwise -- it isn't something crucially important.
# You can further just comment bash commands.
!echo "> ls $dstc2_path"
!ls $dstc2_path

> ls ./dstc2
dstc2-templates.txt  dstc2-tst.jsonlist  resto.sqlite
dstc2-trn.jsonlist   dstc2-val.jsonlist


Let's build our vocabulary.

In [29]:
from deeppavlov import train_model

train_model(vocab_config)

2018-11-14 15:48:55.685 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-trn.jsonlist]
2018-11-14 15:48:55.933 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-val.jsonlist]
2018-11-14 15:48:56.87 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-tst.jsonlist]
[nltk_data] Downloading package punkt to /home/vimary/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vimary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/vimary/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downlo

Vocabulary was built on data and saved to disk.

`save_path = 'vocabs/token.dict'` and component files are saved to `./vocabs/token.dict`.

In [30]:
vocabs_path = './vocabs'

In [31]:
!echo "> ls $vocabs_path"
!ls $vocabs_path

> ls ./vocabs
token.dict


This is the content of the saved "token.dict":

In [32]:
!echo "> head $vocabs_path/token.dict"
!head $vocabs_path/token.dict

> head ./vocabs/token.dict
cheap	391
restaurant	1179
any	259
south	306
address	781
phone	819
number	825
thank	998
you	1055
good	891


##### Using trained component

We can use built vocabulary by initializing it with `build_model`.

We need to add `in` and `out` to component configuration ( to know what are inputs and outputs during prediction ) :
 - **in** -- a list of data fields that are inputs during inference (prediction)
 - **out** -- a list of data fields that are outputs during inference (prediction)

In [33]:
vocab_comp_config['in'] = ['utterance']
vocab_comp_config['out'] = ['utterance_token_indices']

vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]
vocab_config['chainer']['out'] = ['utterance_token_indices']

In [34]:
from deeppavlov import build_model

model = build_model(vocab_config)

2018-11-14 15:50:14.140 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]


Model expects a list of samples (batch) as input.

In [35]:
model(['hi'])

[141]

### Model `gobot_dstc2_simple`

Now let's train a simple goal-oriented bot:

In [36]:
from deeppavlov import build_model, deep_download

"dataset_reader", "dataset_iterator" and "metadata" will be the same as for vocabulary only.

In [59]:
simple_config = {}

simple_config['dataset_reader'] = dstc2_reader_comp_config
simple_config['dataset_iterator'] = dialog_iterator_comp_config
simple_config['metadata'] = {}
simple_config['metadata']['download'] = [
    dstc2_download_config
]

X here is a dict `'x'` containing context 'text', 'intents', db_result', 'prev_resp_act'

Y here is a dict `'y'` containing response 'act' and 'text'

Prediction `'y_predicted'` here will be only 'text'

In [60]:
simple_config['chainer'] = {}
simple_config['chainer']['in'] = ['x']
simple_config['chainer']['in_y'] =['y']
simple_config['chainer']['out'] = ['y_predicted']

The bot consists (`pipe` section) of two components:
- **`default_vocab`** (or DefaultVocabulary) component that 

    - remembers all tokens from user utterances. 
    - `DefaultVocabulary.__call__` method inputs batch of tokens and outputs their indeces.

Vocabulary component will be the same as before, but let's add reference to component using `id` 
- **id** -- reference name for the component

In [61]:
vocab_comp_config = {
    'class_name': 'default_vocab',
    'id': 'token_vocab',
    'load_path': 'vocabs/token.dict',
    'save_path': 'vocabs/token.dict',
    'fit_on': ['x'],
    'level': 'token',
    'tokenizer': {
        'class_name': 'split_tokenizer'
    }
}

Adding vocabulary to chainer:

In [62]:
simple_config['chainer']['pipe'] = []
simple_config['chainer']['pipe'].append(vocab_comp_config)

- **`go_bot`** (or GoalOrientedBot) component that
    - calls `slot_filler` that for user utterance outputs mentioned slots 
        (for example, "i want cheap food" -> {'pricerange': 'cheap'})
    - updates dialog state with `tracker` (DialogStateTracker)
        
          (for example, if old state was {'location': 'north'}, 
          and current slots are {'pricerange': 'cheap'}, 
          then new dialog state will be {'location': 'north', 'pricerange': 'cheap'})
    - converts user utterance in string format (`x`) to tokens with `tokenizer`

          (for example, "hi, i want some cheap food" -> ['hi', ',', 'i', 'want', 'some', 'cheap', 'food'])
    - then embeds the tokens with bag-of-words using `bow_embedder`(if not None) and `word_vocab`

          (for example, "cheap" -> [1, 0, 0, 0, .., 0])
    - embeds the utterance with continuous `embedder` (if not None) as a mean of embeddings of utterance tokens
        
          (for example, "i" -> [0.1231, 0.23423, .., 0.03489])
    - concatenates embeddings and passes it as an input to a recurrent neural network (RNN)
    - trains RNN (with LongShortTermMemory (LSTM) as a core graph) that outputs an action label
    - loads templates (mapping from labels to string) using `template_path` and `template_type` and converts action label to string
        
          (for example, "bye_msg" -> "You are welcome!")
    - fills result string with slot values from dialog state
        
          (for example, if
           dialog state is equal to {'pricerange': 'cheap'}
           and output string is "There are no restaurants in a #pricerange pricerange"
           then the result response will be "There are no restaurants in a cheap pricerange")

In [54]:
bot_comp_config = {
    'class_name': 'go_bot',
    'in': ['x'],
    'in_y': ['y'],
    'out': ['y_predicted'],
    'load_path': 'gobot_dstc2_simple/model',
    'save_path': 'gobot_dstc2_simple/model',
    'word_vocab': None,
    'bow_embedder': None,
    'embedder': None,
    'slot_filler': None,
    'template_path': 'dstc2/dstc2-templates.txt',
    'template_type': 'DualTemplate',
    'database': None,
    'api_call_action': 'api_call',
    'network_parameters': {
      'dense_size': 64,
      'hidden_size': 128,
      'learning_rate': 0.002,
      'attention_mechanism': None
    },
    'tokenizer': {
        'class_name': 'stream_spacy_tokenizer',
        'lowercase': False
    },
    'tracker': {
        'class_name': 'featurized_tracker',
        'slot_names': ['pricerange', 'this', 'area', 'food', 'name']
    },
    'main': True,
    'debug': False
}

This is how we use vocabulary by reference:

In [55]:
bot_comp_config['word_vocab'] = '#token_vocab'

Let's also initialize a bag-of-words one-hot encoder:

In [56]:
bot_comp_config['bow_embedder'] = {
    "class_name": "bow",
    "depth": "#token_vocab.__len__()", # here we referenced __len__() method of token_vocab component
    "with_counts": True
}

Announcing slot filler component.
We assume that slot filler is already trained, and use it by referencing it's config.

In [43]:
from deeppavlov import configs

In [57]:
slot_filler_comp_config = {
    'config_path': configs.ner.slotfill_dstc2
}

Adding slot filler to bot component:

In [58]:
bot_comp_config['slot_filler'] = slot_filler_comp_config

Adding `bot_comp_config` to `pipe`:

In [63]:
simple_config['chainer']['pipe'].append(bot_comp_config)

Neural network (in the bot) is trained in epochs, and needs data in the form of batches.

That is why we are now filling "train" section with training parameters.

- **train** -- training process configuration
     - **train.batch_size** is a number of samples in a batch (feeded to the network during one training step)
     - **train.epochs** is a number of iterations over dataset during training
     - **train.log_every_n_batches** and **train.log_every_n_epochs** control frequency of logging messages
     - **train.metrics** is a list of metrics used to validate our performance
     - **train.val_every_n_epochs** describes how often we calculate metrics on `valid` data split
     - **train.validation_patience** is a number of epochs without metric improvement on `valid` data that we are able to endure =)
     - **validate_best** is a flag that controles

In [64]:
simple_bot_train_config = {
    'batch_size': 4,
    'epochs': 2,
    'log_every_n_batches': -1,
    'log_every_n_epochs': 1,
    'metrics': ['per_item_dialog_accuracy'],
    'val_every_n_epochs': 1,
    'validation_patience': 20,
    'validate_best': False,
    'test_best': False
}

In [65]:
simple_config['train'] = simple_bot_train_config

`train.epochs` is set to '2' for now, if you intend to train a smarter model, you should increase it (a range from 10 to 200 epochs is recommended).

In [49]:
deep_download(slot_filler_comp_config['config_path'])

2018-11-14 15:56:05.21 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-14 15:56:05.26 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-11-14 15:56:05.28 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz to /home/vimary/.deeppavlov/slotfill_dstc2.tar.gz
100%|██████████| 641k/641k [00:00<00:00, 7.34MB/s]
2018-11-14 15:56:05.118 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/vimary/.deeppavlov/slotfill_dstc2.tar.gz archive into /home/vimary/.deeppavlov/models


In [66]:
train_model(simple_config)

2018-11-14 16:02:08.784 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-trn.jsonlist]
2018-11-14 16:02:09.82 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-val.jsonlist]
2018-11-14 16:02:09.162 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-tst.jsonlist]
2018-11-14 16:02:09.258 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:02:09.275 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:02:09.279 INFO in 'deeppavlov.core.data.simple_vocab'

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0162}, "time_spent": "0:00:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 20}}


2018-11-14 16:03:24.44 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 372: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.3805}, "time_spent": "0:01:14"}}


2018-11-14 16:03:44.553 INFO in 'deeppavlov.core.commands.train'['train'] at line 524: New best per_item_dialog_accuracy of 0.4157
2018-11-14 16:03:44.553 INFO in 'deeppavlov.core.commands.train'['train'] at line 526: Saving model
2018-11-14 16:03:44.554 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_simple/model]
2018-11-14 16:03:44.614 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_simple/model.json]


...

2018-11-14 16:04:58.394 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:04:58.405 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:04:58.675 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:04:58.676 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:04:58.676 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-11-14 16:04:59.401 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-14 16:04:59.401 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /ho

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.441}, "time_spent": "0:00:21"}}


2018-11-14 16:05:41.435 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:05:41.437 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 16:05:41.440 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4315}, "time_spent": "0:00:22"}}


2018-11-14 16:05:41.938 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:05:41.948 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:05:42.177 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:05:42.178 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:05:42.178 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-11-14 16:05:42.641 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-14 16:05:42.642 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /ho

Let's comminicate with the resulting bot. "exit" message initiates end of dialogue.

In [67]:
model = build_model(simple_config)

2018-11-14 16:05:42.666 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:05:42.669 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 16:05:42.672 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]
2018-11-14 16:05:43.175 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:05:43.186 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:05:43.439 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/

In [68]:
model(['hi, i want some cheap italian food in the north of town'])

2018-11-14 16:05:44.347 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 392: No database specified.
2018-11-14 16:05:44.347 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [69]:
model(['thanks, bye'])

2018-11-14 16:05:44.362 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 392: No database specified.
2018-11-14 16:05:44.362 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [70]:
model.reset() # resetting dialog context to start a new one

In [71]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: hi, i want some cheap food
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: i want cheap food in the north of town


2018-11-14 16:06:23.404 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 392: No database specified.
2018-11-14 16:06:23.406 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 0 results.


>> Sorry there is no #food restaurant in the north of town.
:: ok, bye


2018-11-14 16:06:27.714 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 392: No database specified.
2018-11-14 16:06:27.716 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 0 results.


>> Sorry there is no #food restaurant in the north of town.
:: exit


The model couldn't fill some slots. For example, #address, #phone, #postcode of a restaurant couldn't be inferred from user utterance. 

A list of available restaurants is required.

### Model `gobot_dstc2_db`

Now let's now add a database with restaurants and train a new model:

Initializing new config:

In [72]:
db_config = copy.deepcopy(simple_config)

db_config['chainer']['pipe'] = []

Creating database component config:

In [73]:
db_comp_config = {
    'class_name': 'sqlite_database',
    'id': 'restaurant_database', 
    'save_path': 'dstc2/resto.sqlite',
    'primary_keys': ['name'],
    'table_name': 'mytable'
}

Adding vocab and database components to pipe:

In [74]:
db_config['chainer']['pipe'].append(vocab_comp_config)
db_config['chainer']['pipe'].append(db_comp_config)

Initializing bot component config:

In [75]:
bot_with_db_comp_config = copy.deepcopy(bot_comp_config)

**WARNING:** Do no forget to change `load_path` and `save_path` in neural network configuration when 
             training a new modification. Otherwise previous model's files will be overwritten.

In [81]:
bot_with_db_comp_config['load_path'] = 'gobot_dstc2_db/model'
bot_with_db_comp_config['save_path'] = 'gobot_dstc2_db/model'

Adding database to bot component config:

In [77]:
bot_with_db_comp_config['database'] = '#restaurant_database'

Addind bot component to pipe:

In [78]:
db_config['chainer']['pipe'].append(bot_with_db_comp_config)

The new model now updates dialog state not only with entity values mentioned by user ("i want cheap food" -> {'pricerange': 'cheap'}), but also with restaurant info taken from sql database of restaurants.

Model has a special action `api_call_action`, which initiates a request to sql database with current dialog state and thus receives info of a single matching restaurant.

So now such slots as #address, #phone and #postcode can be filled in bot responses.

In [84]:
####### Instead of
#
# train_model(db_config)
#
# model = build_model(db_config)
#
####### we can just do

model = train_model(db_config)

2018-11-14 16:10:23.121 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-trn.jsonlist]
2018-11-14 16:10:23.237 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-val.jsonlist]
2018-11-14 16:10:23.315 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-tst.jsonlist]
2018-11-14 16:10:23.409 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:10:23.428 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:10:23.429 WARNING in 'deeppavlov.core.models.seriali

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0156}, "time_spent": "0:00:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 20}}


2018-11-14 16:11:40.538 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 372: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.4258}, "time_spent": "0:01:16"}}


2018-11-14 16:12:02.128 INFO in 'deeppavlov.core.commands.train'['train'] at line 524: New best per_item_dialog_accuracy of 0.4498
2018-11-14 16:12:02.128 INFO in 'deeppavlov.core.commands.train'['train'] at line 526: Saving model
2018-11-14 16:12:02.129 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_db/model]
2018-11-14 16:12:02.187 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_db/model.json]


...

2018-11-14 16:13:17.182 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:13:17.193 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:13:17.471 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:13:17.471 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:13:17.471 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-11-14 16:13:17.957 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-14 16:13:17.957 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /ho

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4675}, "time_spent": "0:00:22"}}


2018-11-14 16:13:59.981 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:13:59.982 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 48: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-11-14 16:13:59.983 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/resto.sqlite.
2018-11-14 16:13:59.985 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 16:13:59.987 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4612}, "time_spent": "0:00:21"}}


2018-11-14 16:14:00.505 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:14:00.516 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:14:00.744 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:14:00.745 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:14:00.746 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-11-14 16:14:01.563 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-14 16:14:01.564 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /ho

In [85]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: HI! I want some cheap food in the north of town


2018-11-14 16:14:21.863 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, give me their phone number
>> The phone number of da vinci pizzeria is 01223 351707.
:: post code too
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: bye then
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: exit


### Model `gobot_dstc2_emb`

Now let's train a goal-oriented bot with fasttext embeddings:

**NOTICE:** YOU NEED TO CONSTRUCT A NEW CONFIG YOURSELF

Ensure that fastText is installed:

In [1]:
!pip install pybind11==2.2.3 
!pip install git+https://github.com/deepmipt/fastText.git#egg=fastText==0.8.22

Initalizing new config:

In [86]:
emb_config = copy.deepcopy(db_config)

emb_config['chainer']['pipe'] = []

Adding vocab and database components to chainer pipe:

In [87]:
emb_config['chainer']['pipe'].append(vocab_comp_config)
emb_config['chainer']['pipe'].append(db_comp_config)

Initalizing embedder component:

In [88]:
embedder_comp_config = {
    'id': 'my_embedder',
    'class_name': 'fasttext',
    'load_path': 'embeddings/dstc2_fastText_model.bin',
    'save_path': 'embeddings/dstc2_fastText_model.bin',
    'dim': 100
}

Adding embedder component to chainer pipe:

In [ ]:
emb_config['chainer']['pipe'].append(embedder_comp_config)

Initializing bot component config:

In [90]:
bot_with_embedder_comp_config = copy.deepcopy(bot_with_db_comp_config)

bot_with_embedder_comp_config['load_path'] = 'gobot_dstc2_emb/model'
bot_with_embedder_comp_config['save_path'] = 'gobot_dstc2_emb/model'

Adding #my_embedder to bot_with_embedder_comp_config:

In [ ]:
bot_with_embedder_comp_config['embedder'] = '#my_embedder'

Adding bot_with_embedder_comp_config to chainer pipe

In [ ]:
emb_config['chainer']['pipe'].append(bot_with_embedder_comp_config)

These are download urls for new required data:

In [93]:
embedder_required_data = {
    'url': 'http://files.deeppavlov.ai/deeppavlov_data/embeddings/dstc2_fastText_model.bin',
    'subdir': 'embeddings'
}

In [ ]:
emb_config['metadata']['download'].append(embedder_required_data)

As far as we are now using embeddings, we added a file named `dstc2_fastText_model.bin` to `metadata.download` section. 

Let's run data loading again.

In [96]:
deep_download(emb_config)

2018-11-14 16:18:12.764 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-14 16:18:12.768 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-11-14 16:18:12.769 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz to /home/vimary/.deeppavlov/slotfill_dstc2.tar.gz
100%|██████████| 641k/641k [00:00<00:00, 12.4MB/s]
2018-11-14 16:18:12.823 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/vimary/.deeppavlov/slotfill_dstc2.tar.gz archive into /home/vimary/.deeppavlov/models
2018-11-14 16:18:12.830 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-14 16:18:12.836 DEBUG in 'urllib3.connectionpool'['connectionpool

In [97]:
model = train_model(emb_config)

2018-11-14 16:18:34.572 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-trn.jsonlist]
2018-11-14 16:18:34.697 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-val.jsonlist]
2018-11-14 16:18:34.781 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-tst.jsonlist]
2018-11-14 16:18:34.877 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:18:34.895 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:18:34.895 WARNING in 'deeppavlov.core.models.seriali

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0128}, "time_spent": "0:00:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 20}}


2018-11-14 16:19:52.204 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 372: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.3778}, "time_spent": "0:01:15"}}


2018-11-14 16:20:13.183 INFO in 'deeppavlov.core.commands.train'['train'] at line 524: New best per_item_dialog_accuracy of 0.4147
2018-11-14 16:20:13.183 INFO in 'deeppavlov.core.commands.train'['train'] at line 526: Saving model
2018-11-14 16:20:13.184 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_emb/model]
2018-11-14 16:20:13.246 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_emb/model.json]


...

2018-11-14 16:21:31.204 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 16:21:31.207 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]
2018-11-14 16:21:31.735 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:21:31.745 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:21:31.981 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:21:31.982 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:21:31.982 INFO in 'deeppavlov

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4532}, "time_spent": "0:00:22"}}


2018-11-14 16:22:16.389 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:22:16.390 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 48: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-11-14 16:22:16.391 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/resto.sqlite.
2018-11-14 16:22:16.392 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/home/vimary/ipavlov/Pilot/examples/tutorials/embeddings/dstc2_fastText_model.bin`]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.449}, "time_spent": "0:00:23"}}


2018-11-14 16:22:17.162 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 16:22:17.165 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]
2018-11-14 16:22:18.8 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:22:18.18 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:22:18.304 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:22:18.305 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:22:18.305 INFO in 'deeppavlov.mo

In [98]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: Hi, i want some cheap food in the north part of town


2018-11-14 16:33:10.125 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: what type of food is there?
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, give me their phone number and address
>> The phone number of da vinci pizzeria is 01223 351707.
:: address, please
>> Sure, da vinci pizzeria is on 20 milton road chesterton.
:: ok, thank you
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: bye
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: exit


## Appendix A
### Additional materials

- [DataFest Presentation (RU)](https://docs.google.com/presentation/d/1PBPQp-wgQ6aRbm3MsuyGYB_TVg2c7Bf89lhdhbMtC2k)
- [CISS Video Lecture](https://youtu.be/uvH1zB7qahI)
- [Video Lecture "Hybrid dialog bot" (RU)](http://www.youtube.com/watch?v=JJCO7eWCy-M&t=331m19s)
- [Video Lecture "What's inside a dialog system?" (RU)](http://www.youtube.com/watch?v=JJCO7eWCy-M&t=259m55s)

## Appendix B
### Model `gobot_dstc2_full`

Now let's train a very smart goal-oriented bot that uses an attention mechanism over input embeddings 

(see https://medium.com/syncedreview/a-brief-overview-of-attention-mechanism-13c578ba9129 for more details):

Initializing new config:

In [99]:
full_config = copy.deepcopy(emb_config)

full_config['chainer']['pipe'] = [
    vocab_comp_config,
    db_comp_config,
    embedder_comp_config
]

Initializing bot component config:

In [100]:
bot_with_attn_comp_config = copy.deepcopy(bot_with_embedder_comp_config)

bot_with_attn_comp_config['load_path'] = 'gobot_dstc2_full/model'
bot_with_attn_comp_config['save_path'] = 'gobot_dstc2_full/model'

Adding attention mechanism to bot:

In [101]:
attention_mechanism_config = {
    'action_as_key': True,
    'depth': 3,
    'hidden_size': 32,
    'max_num_tokens': 100,
    'projected_align': False,
    'type': 'cs_bahdanau'
}

In [102]:
bot_with_attn_comp_config['network_parameters']['attention_mechanism'] = attention_mechanism_config

Adding bot component to pipe:

In [103]:
full_config['chainer']['pipe'].append(bot_with_attn_comp_config)

In [105]:
model = train_model(full_config)

2018-11-14 16:35:51.777 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-trn.jsonlist]
2018-11-14 16:35:51.906 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-val.jsonlist]
2018-11-14 16:35:51.995 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-tst.jsonlist]
2018-11-14 16:35:52.357 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:35:52.375 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 16:35:52.376 WARNING in 'deeppavlov.core.models.seriali

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0096}, "time_spent": "0:02:51", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 20}}


2018-11-14 16:44:07.188 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 372: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.3536}, "time_spent": "0:08:11"}}


2018-11-14 16:46:58.614 INFO in 'deeppavlov.core.commands.train'['train'] at line 524: New best per_item_dialog_accuracy of 0.3589
2018-11-14 16:46:58.614 INFO in 'deeppavlov.core.commands.train'['train'] at line 526: Saving model
2018-11-14 16:46:58.615 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_full/model]
2018-11-14 16:46:58.886 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/gobot_dstc2_full/model.json]


...

2018-11-14 16:54:59.111 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 16:54:59.114 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]
2018-11-14 16:54:59.620 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 16:54:59.630 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 16:54:59.861 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 16:54:59.861 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 16:54:59.862 INFO in 'deeppavlov

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4415}, "time_spent": "0:02:51"}}


2018-11-14 17:00:46.335 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/vocabs/token.dict]
2018-11-14 17:00:46.336 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 48: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-11-14 17:00:46.337 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/resto.sqlite.
2018-11-14 17:00:46.338 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/home/vimary/ipavlov/Pilot/examples/tutorials/embeddings/dstc2_fastText_model.bin`]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4281}, "time_spent": "0:02:54"}}


2018-11-14 17:00:47.70 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-14 17:00:47.73 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]
2018-11-14 17:00:47.576 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-14 17:00:47.586 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-14 17:00:47.810 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/examples/tutorials/dstc2/dstc2-templates.txt]
2018-11-14 17:00:47.811 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-14 17:00:47.811 INFO in 'deeppavlov.m

In [106]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: hi, i want cheap food in the north part of town


2018-11-14 17:05:08.762 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, what type of food is there?
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: i want expensive restaurant now
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: expensive food please
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, bye
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: exit


We've trained for two epochs. For better results use > 15 epochs.

## Appendix C
### Another way of training and infering a component

Let's build response token vocabulary, but do it without using deeppavlov scripts (without `train_model` and `build_model`).

In [126]:
from deeppavlov.core.data.vocab import DefaultVocabulary
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

Initializing a `DefaultVocabulary` class:

In [127]:
y_vocab = DefaultVocabulary(level='token', 
                            load_path='vocabs/y_token.dict', # path is relative to DEEPPAVLOV_ROOT/../download/ 
                            save_path='vocabs/y_token.dict',
                            tokenizer=lambda s_batch: [s.split() for s in s_batch])

Important methods of any trained component are:

   - **\_\_init\_\_(self, *args, *\*kwargs)**
     - intializes a class instance

   - **fit(self, data, *args)** or **train_on_batch(self, batch, *args)**
     - fits on full data or makes one training step on a batch of data

   - **\_\_call\_\_(self, batch, \*\*kwargs)**
     - makes prediction (or infers) for each sample in a batch

Getting batches of data:

In [ ]:
data = DSTC2DatasetReader().read(data_path="tmp/my_download_of_dstc2")
data_samples = DialogDatasetIterator(data, seed=1443, shuffle=True).get_instances(data_type='all')
x_list, y_list = data_samples

Building vocabulary using y batches (`y_list` contains bot responses):

In [129]:
y_vocab.fit(y_list)

Infering from (using) built vocabulary:

In [130]:
y_vocab(['is', 'the', 'of', 'restaurant', 'hi'])

[43, 3, 26, 5, 0]

To call a model `y_vocab(batch)` is the same as to call a \_\_call\_\_ method `y_vocab.__call__(batch)`!

In [198]:
y_vocab(['hi']) == y_vocab.__call__(['hi']) == [141]

False